In [188]:
import psycopg2
from yaml import load as yaml_load, FullLoader
from pandas import DataFrame, to_datetime, Timedelta
from sys import path
path.insert(0,'../../python')
from google_api import *
from map_utils import *
import pandas as pd

In [75]:
begin = "ROBERT BOSCH MEXICO Santa Fe CDMX"
end = "Orizaba 32 CDMX"

# geocode_result = gmaps.geocode(begin)
# bosch_location = get_geocode_locations(geocode_result)[0]
bosch_location = [19.3675809, -99.26231600000001]

# geocode_result = gmaps.geocode(end)
# orizaba_location = get_geocode_locations(geocode_result)[0]
orizaba_location = [19.4225152, -99.1606875]

In [37]:
db = yaml_load(open('../../auth/db.yaml'), Loader=FullLoader)
conn = psycopg2.connect(database = db['pg_database'],
                        user = db['pg_user'],
                        password = db['pg_password'],
                        host = db['pg_host'],
                        port = db['pg_port'])

print ("Opened database successfully")



Opened database successfully


In [38]:
query = '''
SELECT
   *
FROM
   pg_catalog.pg_tables
WHERE
   schemaname != 'pg_catalog'
AND schemaname != 'information_schema';
'''

In [39]:
with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    table = DataFrame(rows, columns=colnames)
usuarios_btc = table.copy()

In [41]:
query = '''
select schemaname, viewname from pg_catalog.pg_views
where schemaname NOT IN ('pg_catalog', 'information_schema')
order by schemaname, viewname;
'''

with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    table = DataFrame(rows, columns=colnames)

table

,schemaname,viewname
0,public,client_clasification_per_date
1,public,client_clasification_per_month
2,public,client_view
3,public,geography_columns
4,public,geometry_columns
5,public,payment_card_view
6,public,raster_columns
7,public,raster_overviews
8,public,reservation_btb_view
9,public,reservation_btb_view_dev


In [42]:
table_name = 'stations_view'

In [66]:
query = f"SELECT * FROM {table_name} LIMIT {1000};"

with conn.cursor() as cur:
    cur.execute(query)
    rows = cur.fetchall()
    colnames = [desc[0] for desc in cur.description]
    table = DataFrame(rows, columns=colnames)

table

,station_id,route_id,route_name,station_name,type_of,order,station_latitude,station_longuitude
0,23,180,Santa fe - Izcalli,Edificio Telefónica,PU,2,19.3589570000000002,-99.2732799999999997
1,574,319,ITESM R-V Ida,Ruta V - Eje 8,PU,2,19.3666605999999994,-99.1690940000000012
2,320,215,Santa fe - Nápoles,Benavides Nápoles,DO,3,19.3892890000000016,-99.1760340000000014
3,146,138,Lindavista - Santa fe,Superama Lindavista,PU,1,19.4902179999999987,-99.1337330000000065
4,594,325,ITESM R-T Ida,Ruta T - De las flores,PU,5,19.3407060000000008,-99.2331850000000060
...,...,...,...,...,...,...,...,...
991,454,293,ITESM R-H Ida,Ruta H - Monte Chimborazo,PU,12,19.4244510999999989,-99.2225789999999961
992,131,133,Santa fe - Coacalco,Sam's Santa Fe,PU,1,19.3551207851673475,-99.2763871059086114
993,215,244,Auditorio - Coacalco,Auditorio (Pdte. Intercontinental),PU,1,19.4269449999999999,-99.1943229999999971
994,114,335,Zona norte - Auditorio,Multiplaza Arboledas,PU,1,19.5466020000000000,-99.2100180000000051


In [67]:
import numpy as np
def filter_coordinates(df,bounds):
    def filter_coords(coords, bounds):
        return list(map(lambda x: ispointinside(x, bounds) if not np.isnan(x[0]) else False, coords))
    df['in_bounds'] = filter_coords(df[['station_latitude', 'station_longuitude']].astype(float).values, bounds)

In [68]:
bounds = [
    [ 19.114029215761995,-99.87945556640625],
    [19.114029215761995,-98.778076171875],
    [19.619305073651013,-98.778076171875],
    [19.619305073651013,-99.87945556640625],
    [19.114029215761995,-99.87945556640625]
          ]

In [61]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 8 columns):
station_id            996 non-null int64
route_id              996 non-null int64
route_name            996 non-null object
station_name          996 non-null object
type_of               996 non-null object
order                 996 non-null int64
station_latitude      996 non-null object
station_longuitude    996 non-null object
dtypes: int64(3), object(5)
memory usage: 62.4+ KB


In [69]:
table = table.loc[~np.isnan(table.station_longuitude.astype(float))]
filter_coordinates(table,bounds)

In [107]:
origin = bosch_location
dest = orizaba_location

In [183]:
routes_id = table.route_id.unique()
distances = []
for i in routes_id:
    route = table.loc[table.route_id == i].sort_values('order')
    up_stations = route.loc[route.type_of == 'PU']
    down_stations = route.loc[route.type_of == 'DO']
    up_coords = up_stations[['station_latitude','station_longuitude'] ].astype(float).values
    down_coords = down_stations[['station_latitude','station_longuitude'] ].astype(float).values
    up_distances = getDistance2(origin,up_coords)
    down_distances = getDistance2(dest,down_coords)
    if len(down_distances) and len(down_distances):
        distance = up_distances.min() + down_distances.min()
        distances.append(distance)
    else:
        distances.append(np.inf)
distances = np.asarray(distances)

table.loc[table.route_id == routes_id[distances.argmin()]].sort_values('order')

,station_id,route_id,route_name,station_name,type_of,order,station_latitude,station_longuitude,in_bounds
321,23,262,Santa fe - Reforma,Edificio Telefónica,PU,1,19.3589570000000002,-99.2732799999999997,True
204,303,262,Santa fe - Reforma,German Centre Mexico,PU,2,19.3667659999999984,-99.2608740000000012,True
519,350,262,Santa fe - Reforma,Glorieta de la palma,DO,3,19.4285414694348795,-99.1630721779098394,True


In [184]:
origin = orizaba_location
dest = bosch_location

In [185]:
routes_id = table.route_id.unique()
distances = []
for i in routes_id:
    route = table.loc[table.route_id == i].sort_values('order')
    up_stations = route.loc[route.type_of == 'PU']
    down_stations = route.loc[route.type_of == 'DO']
    up_coords = up_stations[['station_latitude','station_longuitude'] ].astype(float).values
    down_coords = down_stations[['station_latitude','station_longuitude'] ].astype(float).values
    up_distances = getDistance2(origin,up_coords)
    down_distances = getDistance2(dest,down_coords)
    if len(down_distances) and len(down_distances):
        distance = up_distances.min() + down_distances.min()
        distances.append(distance)
    else:
        distances.append(np.inf)
distances = np.asarray(distances)

table.loc[table.route_id == routes_id[distances.argmin()]].sort_values('order')

,station_id,route_id,route_name,station_name,type_of,order,station_latitude,station_longuitude,in_bounds
479,350,261,Reforma - Santa Fe,Glorieta de la palma,PU,1,19.4285414694348795,-99.1630721779098394,True
9,303,261,Reforma - Santa Fe,German Centre Mexico,DO,2,19.3667659999999984,-99.2608740000000012,True
650,23,261,Reforma - Santa Fe,Edificio Telefónica,DO,3,19.3589570000000002,-99.2732799999999997,True


In [45]:
from datetime import datetime
import pytz
from tzlocal import get_localzone # $ pip install tzlocal

utc_dt = datetime(2009, 7, 10, 18, 44, 59, 193982, tzinfo=pytz.utc)
print(utc_dt.astimezone(get_localzone()))

2009-07-10 13:44:59.193982-05:00


In [35]:
print(table.departure_time[0].astimezone(get_localzone()))

2020-01-21 16:30:00-06:00


In [153]:
departures = ["09:00",
"09:40",
"14:30",
"14:35",
"14:50",
"15:30",
"16:30",
"16:40",
"17:00",
"17:05",
"17:10",
"17:15",
"17:20",
"18:20",
"19:00",
"19:10",
"19:40",
"19:50",
"20:00"]


In [170]:
arrivals = ["10:00",
"10:50",
"16:10",
"15:35",
"15:50",
"16:40",
"17:30",
"17:40",
"18:00",
"18:05",
"18:10",
"18:15",
"18:20",
"19:20",
"20:00",
"20:10",
"20:40",
"20:50",
"21:10"]

In [171]:
from datetime import datetime, timedelta

In [189]:
durations = []
for departure,arrival in zip(departures, arrivals):
    hours, minutes =  [int(x) for x in departure.split(':')]
    departure_time = datetime(2020,2,24, hours,minutes,0)
    hours, minutes =  [int(x) for x in arrival.split(':')]
    arrival_time = datetime(2020,2,24, hours,minutes,0)
    td = arrival_time - departure_time
    duration = ':'.join(str(td).split(':')[:2])
    durations.append(duration)

In [193]:
df = pd.DataFrame({'departure':departures,
                  'duration':durations,
                  'arrival':arrivals})
df.to_csv('../../data/Orizaba 32/urbvan_way1.csv')

,departure,duration,arrival
0,09:00,1:00,10:00
1,09:40,1:10,10:50
2,14:30,1:40,16:10
3,14:35,1:00,15:35
4,14:50,1:00,15:50
5,15:30,1:10,16:40
6,16:30,1:00,17:30
7,16:40,1:00,17:40
8,17:00,1:00,18:00
9,17:05,1:00,18:05


In [197]:
departures = [
"07:00",
"07:10",
"08:05",
"08:10",
"09:00",
"09:10",
"09:20",
"09:25",
"09:30",
"09:40",
"09:50",
"16:50"]


In [198]:
arrivals = [
"08:00",
"08:10",
"09:05",
"09:10",
"10:00",
"10:10",
"10:20",
"10:25",
"10:30",
"10:40",
"10:50",
"18:00"]


In [199]:
durations = []
for departure,arrival in zip(departures, arrivals):
    hours, minutes =  [int(x) for x in departure.split(':')]
    departure_time = datetime(2020,2,24, hours,minutes,0)
    hours, minutes =  [int(x) for x in arrival.split(':')]
    arrival_time = datetime(2020,2,24, hours,minutes,0)
    td = arrival_time - departure_time
    duration = ':'.join(str(td).split(':')[:2])
    durations.append(duration)

In [200]:
df = pd.DataFrame({'departure':departures,
                  'duration':durations,
                  'arrival':arrivals})
df.to_csv('../../data/Orizaba 32/urbvan_way2.csv')

In [201]:
df

,departure,duration,arrival
0,07:00,1:00,08:00
1,07:10,1:00,08:10
2,08:05,1:00,09:05
3,08:10,1:00,09:10
4,09:00,1:00,10:00
5,09:10,1:00,10:10
6,09:20,1:00,10:20
7,09:25,1:00,10:25
8,09:30,1:00,10:30
9,09:40,1:00,10:40
